## Secured DC PF

$$ \rho^* = {\arg\,\min_{\rho, \vec{f}}}\, \rho\\
s.t.\\
\vec{f} = \mathbf{DA\hat{B}^{-1}} \vec{p}\\
|\vec{f}| \preceq \rho \overline{\vec{f}}$$

$$ \rho^* = \arg\,\min_{\rho, \vec{f}}\,\max_{\vec{\beta},\vec{u}}\, \rho\\
s.t.\\
\vec{f_i} = \mathbf{DA\hat{B}^{-1}} \vec{p} + (\mathbf{D - DA\hat{B}^{-1}A^TD})\vec{\beta}\\
|\vec{f}| \preceq \rho \overline{\vec{f}}$$

ie with $ dim(\vec{u}) = dim(\vec{\beta}) = dim(\vec{\alpha})$
$$ \rho^* = \arg\,\min_{\rho}\, \rho \\
s.t. \\
\vec{F_i} = \mathbf{DA\hat{B}^{-1}} \vec{p} + (\mathbf{D - DA\hat{B}^{-1}A^TD})\vec{\beta_i} \ \forall i\\
\rho \ge \alpha_i \ \forall i\\
|\vec{F_i}| \preceq \alpha_i \overline{\vec{f}} \ \forall i \\
0 \preceq \vec{\alpha} \preceq \vec{u}M \\
|\vec{f}| \preceq (1-\vec{u})M \\
|\vec{\beta}| \preceq \vec{u}M \\
u_l \in \{0,1\} \\
\sum_l{u_l} \le 1
$$


In [ ]:
function create_dc_matrix(g)
    A = incidence_matrix(g; oriented = true)'
    D = spdiagm([e_index_for(g,e).b for e in edges(g)])
    B = A'*D*A
    B_inv = inv(Matrix(B + fill(1/nv(g), nv(g), nv(g))))
    p_to_f = D*A*B_inv
    δ_to_f = D - p_to_f*A'*D
    A, D, B_inv, p_to_f, δ_to_f
end

create_dc_matrix (generic function with 1 method)

In [ ]:
function secured_dc_pf_optim(g, contingencies)
    model = direct_model(Gurobi.Optimizer())
    set_silent(model)

    A, D, B_inv, p_to_f, δ_to_f = create_dc_matrix(g)

    big_M = 100

    p = [g[label_for(g, v)] for v in vertices(g)]
    p_max = [e_index_for(g, e).p_max for e in edges(g)]
    
    outages = collect(contingencies)
    push!(outages, 0)
    nb_c = length(outages)

    @variable(model, β[1:nb_c, 1:ne(g)])
    @constraint(model, [i=1:nb_c],  β[i,:] .≤ [j==outages[i] ? big_M : 0 for j in 1:ne(g)])
    @constraint(model, [i=1:nb_c], -β[i,:] .≤ [j==outages[i] ? big_M : 0 for j in 1:ne(g)])

    @variable(model, flows[1:nb_c, 1:ne(g)])
    @constraint(model, [i=1:nb_c], flows[i,:] .== p_to_f * p + δ_to_f * β[i,:])
    @constraint(model, [i=1:nb_c-1], flows[i, outages[i]] == 0) # i=nb_c : base case
    
    @variable(model, α[1:nb_c])
    @constraint(model, [i=1:nb_c], flows[i,:] .≤ α[i].*p_max)
    @constraint(model, [i=1:nb_c], -α[i].*p_max .≤ flows[i,:])

    @variable(model, α_max)
    @constraint(model, [i=1:nb_c], α .≤ α_max)

    @objective(model, Min, α_max)
    model
end

secured_dc_pf_optim (generic function with 1 method)

## Secured DC PF 2

$$ \rho^* = \arg\,\max_{\vec{\beta},\vec{u}}\, \min_{\rho}\rho\\
s.t.\\
\vec{f} = \mathbf{DA\hat{B}^{-1}} \vec{p} + (\mathbf{D - DA\hat{B}^{-1}A^TD})\vec{\beta}\\
|\vec{f}| \preceq \rho \overline{\vec{f}} \\
(1-\vec{u}) \odot \vec{\beta} = 0 \\
u_l \in \{0,1\} \\
\sum_l{u_l} \le 1

$$

$$ \rho^* = \arg\,\max_{\vec{w},\vec{u}, \vec{\beta},\rho}\, \rho \\
s.t. \\
\vec{f} = \mathbf{DA\hat{B}^{-1}} \vec{p} + (\mathbf{D - DA\hat{B}^{-1}A^TD})\vec{\beta} \\
|\vec{f}| \preceq (1-\vec{u})M \\
|\vec{f} - \rho \overline{\vec{f}}| \preceq (1-\vec{w})M\\
|\vec{\beta}| \preceq \vec{u}M \\
u_l \in \{0,1\} \\
\sum_l{u_l} \le 1
$$

$$
w_l \in \{0,1\} \qquad \forall l \\
\sum_{l}{w_l}=1
$$

In [ ]:
function secured_dc_pf_optim_2(g)
    model = Model(() -> Gurobi.Optimizer(env))
    set_silent(model)

    A = incidence_matrix(g; oriented = true)'
    D = spdiagm([e_index_for(g,e).b for e in edges(g)]) #TODO: check consistency
    B = A'*D*A
    B_inv = inv(Matrix(B + spdiagm(fill(1/nv(g), nv(g)))))
    p_to_f = D*A*B_inv
    δ_to_f = D - p_to_f*A'*D
    big_M = 100

    p = [g[label_for(g, v)] for v in vertices(g)]
    p_max = [e_index_for(g, e).p_max for e in edges(g)]
    
    @variable(model, flows[1:ne(g)])
    @variable(model, β[1:ne(g)])
    @variable(model, ρ)
    # @variable(model, 0 ≤ u[i=1:ne(g)] ≤ 1)
    @variable(model, u[1:ne(g)], Bin)
    @variable(model, w[1:ne(g)], Bin)
    # @variable(model, 0 ≤ w[i=1:ne(g)] ≤ 1)
    
    @constraint(model, c_flows , flows == p_to_f * p + δ_to_f * β)

    @constraint(model,  flows ≤ ρ.*p_max)
    @constraint(model, -flows ≤ ρ.*p_max)
    
    @constraint(model,  flows .≤ big_M.*(1 .- u))
    @constraint(model, -flows .≤ big_M.*(1 .- u))
    
    @constraint(model, ρ .- flows .≤ big_M.*(1 .- w))
    @constraint(model, flows .- ρ .≤ big_M.*(1 .- w))

    @constraint(model,  β ≤ big_M .* u)
    @constraint(model, -β ≤ big_M .* u)

    @constraint(model, sum_u, sum(u) == 1)
    @constraint(model, sum_w, sum(w) == 1)

    @objective(model, Max, ρ)
    model
end

secured_dc_pf_optim_2 (generic function with 1 method)

In [ ]:
sys = System(joinpath("data", "case118.m"))
# sys = System(joinpath("data", "case1354pegase.m"))
g = network2graph(sys)
add_constraint(g, b->b.p_max=1)

┌ Info: Correcting vm in bus 19 to 0.962 to match generator set-point
└ @ PowerSystems /Users/benoitjeanson/.julia/packages/PowerSystems/mjN6j/src/parsers/pm_io/matpower.jl:242
┌ Info: Correcting vm in bus 32 to 0.963 to match generator set-point
└ @ PowerSystems /Users/benoitjeanson/.julia/packages/PowerSystems/mjN6j/src/parsers/pm_io/matpower.jl:242
┌ Info: Correcting vm in bus 34 to 0.984 to match generator set-point
└ @ PowerSystems /Users/benoitjeanson/.julia/packages/PowerSystems/mjN6j/src/parsers/pm_io/matpower.jl:242
┌ Info: Correcting vm in bus 92 to 0.99 to match generator set-point
└ @ PowerSystems /Users/benoitjeanson/.julia/packages/PowerSystems/mjN6j/src/parsers/pm_io/matpower.jl:242
┌ Info: Correcting vm in bus 103 to 1.01 to match generator set-point
└ @ PowerSystems /Users/benoitjeanson/.julia/packages/PowerSystems/mjN6j/src/parsers/pm_io/matpower.jl:242
┌ Info: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 1 from -360.0 t

In [ ]:
model = secured_dc_pf_optim_2(g)
optimize!(model)
# display(latex_formulation(model))
println(value(model[:ρ]))
println(value.(model[:u]))
println(value.(model[:w]))
# println(value.(model[:flows]))
print(solution_summary(model))

4.823535445870746
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

# Work in progress

$$
\max_{\vec{\beta}, \vec{u}}{\rho} \\
\begin{split}
    \vec{g} = \vec{f} -\mathbf{DA}\mathbf{\hat{B}}^{-1}\vec{p} - (\mathbf{D} -\mathbf{DA}\mathbf{\hat{B}}^{-1}\mathbf{A}^T\mathbf{D})(\vec{\beta} + \mathbf{T}\vec{\gamma}) & \rightarrow \vec{\nu} \\
    \vec{f} - (1-\vec{u})M \preceq 0 & \rightarrow \vec{\lambda_{f pos}}\\
    - \vec{f} - (1-\vec{u})M \preceq 0 & \rightarrow  \vec{\lambda_{f neg}}\\
    \vec{\beta} - \vec{u}M \preceq 0 & \rightarrow  \vec{\lambda_{\beta pos}}\\
    -\vec{\beta} - \vec{u}M \preceq 0 & \rightarrow \vec{\lambda_{\beta neg}}\\
    \vec{u} -1 \preceq 0 \ & \rightarrow \vec{\lambda_{u pos}} \\
    -\vec{u} \preceq 0 \ l & \rightarrow  \vec{\lambda_{u neg}} \\
    -1 + \sum_l{u_l} \le 0 & \rightarrow  \lambda_{u sum}
\end{split}
$$

It is a LP, for which the dual function is known.
$$
\max_{\vec{x}} \vec{c} \odot \vec{x} \\
s.t. \\
\mathbf{A} \vec{x} = \vec{b} \\
\vec{x} \preceq 0
$$

Dual
$$
\min_{\vec{\nu}, \vec{\lambda}} -\vec{b} \odot \vec{x} \\
s.t. \\
\vec{c} + \vec{\lambda} + \mathbf{A}^T\vec{\nu} = 0 \\
\vec{\lambda} \succeq 0
$$


